In [171]:
import os,gc
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# 读取数据
# 训练集
data_1 =  pd.read_excel('./data/train/data1/Part_1.xlsx')
data_2 =  pd.read_excel('./data/train/data2/Part_2.xlsx')
data_3 =  pd.read_excel('./data/train/data3/Part_3.xlsx')
data_4 =  pd.read_excel('./data/train/data4/Part_4.xlsx')
data_5 =  pd.read_excel('./data/train/data5/Part_5.xlsx')
data_6 =  pd.read_excel('./data/train/data6/Part_6.xlsx')
# data_7 =  pd.read_excel('./data/train/data7/Part_7.xlsx')
# 合并
train_df = pd.concat([data_1,data_2,data_3,data_4,data_5,data_6],ignore_index=True)

# 测试集
test_df = pd.read_excel('./data/test/test.xlsx')

# 合并训练集，测试集
train_df['is_test'] = 0
test_df['is_test'] = 1
df_all = pd.concat([train_df,test_df],axis=0,ignore_index=True)

# # 文本清洗
# def clean_text(text):
#     # 去除标点符号和数字
#     # text = re.sub(r'[^\u4e00-\u9fa5]+', '', text)
#     # 去掉文本中的空格
#     text = text.replace(' ','') 
#     text = text.replace('...',',') 
#     # text = text.replace('【',' ') 
#     # text = text.replace('】',',')
#     # text = text.replace('[',' ') 
#     # text = text.replace(']',',')

#     return text
# df_all['text'] = df_all['text'].apply(lambda x:clean_text(x))

#文本特征
tfidf = TfidfVectorizer(max_features=16000)
tfidf_df = tfidf.fit_transform(df_all['text'])
tfidf_feat = pd.DataFrame(tfidf_df.toarray(),columns=tfidf.get_feature_names_out())

# #图像特征
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
# ])

In [172]:
# 构造
tfidf_feat['id'] = df_all['id']
tfidf_feat['label'] = df_all['label']
tfidf_feat['is_test'] = df_all['is_test']
df_all = tfidf_feat
df_all = df_all.fillna(0)

# 分离训练集，测试集
train_df = df_all[df_all['is_test']==0]
test_df = df_all[df_all['is_test']==1]

feats = [col for col in test_df if col not in ['id','label','is_test']]                                                                                                                                                                                      
print('feats_num:',len(feats))

del tfidf_df,tfidf_feat,df_all

feats_num: 16000


In [173]:
#内存压缩
import time
pd.set_option('mode.chained_assignment', None)
def reduce_mem_usage(df):
    starttime = time.time()
    numerics = ['int16','int32','int64','float16','float32 ','float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[ col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[ col].max()
            if pd.isnull(c_min) or pd.isnull(c_max):
                continue
            if str(col_type)[:3] == 'int' :
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8 ).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64 ).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype( np.float64)
    end_mem = df.memory_usage( ).sum() / 1024**2
    print('Start Memory usage is: {:.2f} MB'.format(start_mem))
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

reduce_mem_usage(train_df)
reduce_mem_usage(test_df)

Start Memory usage is: 2986.34 MB
Memory usage after optimization is: 746.75 MB
Decreased by 75.0%
Start Memory usage is: 1425.77 MB
Memory usage after optimization is: 356.52 MB
Decreased by 75.0%


In [174]:
import gc
# from vote_mode import get_model
from sklearn.metrics import f1_score,accuracy_score
from sklearn.model_selection import StratifiedKFold,KFold

import joblib
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# 定义模型
def get_model():

    # rf = RandomForestClassifier()
    # clf2 = MultinomialNB(alpha=0.01)
    # clf = MultinomialNB(alpha=0.0225)
    # cat = CatBoostClassifier()

    rg = LogisticRegression(penalty='l2',tol=1e-4)
    sgd = SGDClassifier(n_iter_no_change = 10,max_iter=1000, tol=1e-5, loss="modified_huber", random_state=67654)
    lgb = LGBMClassifier(n_estimators=200,num_leaves=256,max_depth=10)

    weights = [0.08, 0.9, 0.02]
    ensemble = VotingClassifier(estimators=[('rg', rg),
                                            ('sgd', sgd),
                                            ('lgb', lgb)],
                                weights=weights, voting='soft', n_jobs=-1)
    return ensemble

def cross_validate_score(model, train_df, y, cv, test_df):
    val_scores = []
    test_preds = np.zeros((test_df.shape[0],))
    oof_preds = np.zeros((train_df.shape[0],))


    for fold, (train_idx, val_idx) in enumerate(cv.split(train_df, y)):

        X_train, y_train = train_df.iloc[train_idx].reset_index(drop=True), y.iloc[train_idx].reset_index(drop=True)
        X_val, y_val = train_df.iloc[val_idx].reset_index(drop=True), y.iloc[val_idx].reset_index(drop=True)
        
        model.fit(X_train, y_train)
        
        val_probs = model.predict_proba(X_val)[:, 1]  
        val_preds = np.where(val_probs>0.5,1,0)

        val_score = f1_score(val_preds, y_val, average='weighted')  
        print(f'Fold {fold+1}: score = {val_score:.5f}')
        
        val_scores.append(val_score)
        
        oof_preds[val_idx] = val_probs  

        test_preds += model.predict_proba(test_df)[:, 1] / cv.get_n_splits()  

    mean_val_score = np.mean(val_scores)
    std_val_score = np.std(val_scores)
    print(f'Mean Validation score: {mean_val_score:.7f}')
    print(f'Std Validation score: {std_val_score:.7f}')

    del X_train,y_train,X_val,y_val,model
    
    gc.collect()

    return val_scores, test_preds, oof_preds

# 训练
ensemble_model = get_model()
target_col = 'label'
cv = KFold(5, shuffle=True, random_state=9999)  #StratifiedGroupKFold
val_scores, test_preds, oof_preds = cross_validate_score(ensemble_model, train_df[feats], train_df[target_col], cv, test_df[feats])

MemoryError: Unable to allocate 2.33 GiB for an array with shape (19566, 16000) and data type float64

In [31]:
# 融合
# predictions_prob_dl_oof =  pd.read_csv('predictions_prob_dl_oof.csv')
predictions_prob_dl_test =  pd.read_csv('predictions_prob_dl_test_0.9003.csv')
predictions_prob_dl_test2 =  pd.read_csv('predictions_prob_dl_test_0.8993.csv')

# preds = predictions_prob_dl['target']*0.8 + test_preds*0.2

In [ ]:
# 模型权重搜索
scores = []; Weights = []
model_best_m = 0
best_score = 0
for m in np.arange(0.1,0.9,0.01):
    enb_pre = predictions_prob_dl_oof['target']*m + oof_preds*(1-m)
    score = f1_score(np.where(enb_pre>0.5,1,0), train_df[target_col], average='weighted')
    scores.append(score)
    Weights.append(m)
    if score > best_score:
        best_score = score
        model_best_m = m
print(f'best_score : {best_score}   best_m : {model_best_m}')

In [ ]:
# # 指标权重搜索
scores = []; Weights = []
f1_best_m = 0
best_score = 0
for m in np.arange(0.1,0.9,0.01):
    pres = predictions_prob_dl_oof['target']
    score = f1_score(np.where(pres>m,1,0), train_df[target_col], average='weighted')
    scores.append(score)
    Weights.append(m)
    if score > best_score:
        best_score = score
        f1_best_m = m
print(f'best_score : {best_score}   best_m : {f1_best_m}')

In [ ]:
# 融合
# 第一层
pres = predictions_prob_dl_test['target']*0.6 + predictions_prob_dl_test2['target']*0.4
# 第二层
final_pres = pres*0.47 + test_preds*0.53
test_label = np.where(final_pres>0.5,1,0)
# 保存
predictions = pd.DataFrame({'id':test_df['id'],'target':test_label})
predictions.to_csv('predictions.csv',index=False)

In [ ]:
# 投票融合
predictions1 =  pd.read_csv('./ensemble_sub/predictions_bert_enb_prob_test.csv')
predictions2 =  pd.read_csv('./ensemble_sub/predictions_bert_lgb_test.csv')
predictions3 =  pd.read_csv('./ensemble_sub/predictions_prob_dl_test_0.8993.csv')
predictions3['target'] = np.where(predictions3['target']>0.4,1,0)
predictions4 =  pd.read_csv('./ensemble_sub/predictions_prob_dl_test_0.9003.csv')
predictions4['target'] = np.where(predictions4['target']>0.4,1,0)
predictions5 =  pd.read_csv('./ensemble_sub/predictions_score0.8562.csv')
predictions6 =  pd.read_csv('./ensemble_sub/predictions_score0.9061.csv')
predictions7 =  pd.read_csv('./ensemble_sub/predictions_score0.9171.csv')
predictions8 =  pd.read_csv('./ensemble_sub/predictions_score0.9275.csv')
predictions9 =  pd.read_csv('./ensemble_sub/predictions_score0.9294.csv')
predictions10 =  pd.read_csv('./ensemble_sub/predictions_score0.9306.csv')
predictions11 =  pd.read_csv('./ensemble_sub/predictions_score0.9323.csv')
predictions12 =  pd.read_csv('./ensemble_sub/predictions_score0.9387.csv')
predictions13 =  pd.read_csv('./ensemble_sub/predictions_score0.9402.csv')
predictions14 =  pd.read_csv('./ensemble_sub/predictions_score0.9411.csv')
predictions15 =  pd.read_csv('./ensemble_sub/predictions_score0.9418.csv')
predictions16 =  pd.read_csv('./ensemble_sub/predictions_score0.9422.csv')
predictions17 =  pd.read_csv('./ensemble_sub/predictions_score0.934.csv')
predictions18 =  pd.read_csv('./sub/predictions_0.9038.csv')
predictions19 =  pd.read_csv('./sub/predictions_0.9042.csv')
predictions20=  pd.read_csv('./sub/predictions_0.9043.csv')
predictions21=  pd.read_csv('./sub/predictions_0.9061.csv')
pres_df = pd.DataFrame({
                        'predictions5':predictions5['target'],'predictions6':predictions6['target'],
                        'predictions7':predictions7['target'],'predictions8':predictions8['target'],
                        'predictions9':predictions9['target'],'predictions10':predictions10['target'],
                        'predictions11':predictions11['target'],'predictions12':predictions12['target'],
                        'predictions15':predictions15['target'],'predictions16':predictions16['target'],
                        'predictions17':predictions17['target'],'predictions18':predictions18['target'],
                        'predictions19':predictions19['target'],'predictions20':predictions20['target'],
                        'predictions21':predictions21['target']                                            })

preds = pres_df.mode(axis=1)[0].values

# 保存
predictions = pd.DataFrame({'id':test_df['id'].values,'target':preds.astype('int')})
predictions.to_csv('predictions.csv',index=False)

In [168]:
predictions3

,id,target
0,402,0.718481
1,30953,0.998661
2,21387,0.996258
3,39304,0.999024
4,25836,0.019439
...,...,...
11672,9635,0.999719
11673,5435,0.994297
11674,8040,0.002317
11675,25891,0.074487
